In [1]:
import requests
import pandas as pd
from urllib.request import Request, urlopen
from io import BytesIO
from api_secrets import *
import numpy as np
import plotly.express as px
import datetime

In [2]:
users_tt_url = "https://app.timetastic.co.uk/api/users"
users_eh_url = Request('https://api.everhour.com/team/users', headers=eh_key)
clients_url = Request('https://api.everhour.com/clients', headers=eh_key)
project_url = Request('https://api.everhour.com/projects', headers=eh_key)
bankhol_url = "https://app.timetastic.co.uk/api/publicholidays?year=2023"
hol_url = "https://app.timetastic.co.uk/api/holidays?Start=2023-04-20&End=2023-12-31"

In [3]:
# for person dim table

usersTT = requests.get(users_tt_url, headers=tt_key)
users_eh = urlopen(users_eh_url).read()

users_eh = pd.read_json(BytesIO(users_eh))
users_eh = users_eh[['id','name','headline','capacity']]
users_eh = users_eh.rename(columns={'id':'everhour id'})

jobs_df = pd.read_csv('datasets\job_titles.csv')
day_rates = pd.read_csv('datasets\day_rates.csv')

person_df = pd.read_json(usersTT.text)
person_df.insert(1, 'name', person_df.firstname + ' ' + person_df.surname)
person_df.insert(3, 'FTE', users_eh.capacity/60/60/40)
person_df = person_df.rename(columns={'id':'timetastic id'})

person_df = person_df.merge(users_eh,how = 'left', on='name')

person_df = person_df[['everhour id', 'timetastic id', 'name', 'FTE', 'departmentId', 'startDate', 'endDate']]
person_df = (person_df.fillna(0))
person_df = person_df.astype({"everhour id":'int'}) 
person_df.insert(0, 'Concatenated ID', person_df['everhour id'].astype(str) + person_df['timetastic id'].astype(str))

person_df = person_df.merge(jobs_df,how = 'left', on=['name'])
person_df = person_df.merge(day_rates,how = 'left', on=['name'])

person_df = person_df[['Concatenated ID','everhour id', 'timetastic id', 'name', 'headline', 'FTE', 'departmentId', 'departmentName', 'startDate', 'endDate','Day Rate']]
person_df.insert(len(person_df.columns), 'Valid From', pd.to_datetime('2023-03-30').date())
person_df.insert(len(person_df.columns), 'Valid To', pd.to_datetime('2262-04-11').date())
#display(person_df)

In [4]:
# For project dim table
# Project information given by RP

proj_df = pd.read_csv('datasets\projects.csv')
leads = pd.read_csv('datasets\project_leads.csv')
proj_df = proj_df.merge(leads,how = 'left', on=['Client', 'Project Name'])
proj_df = proj_df[['Client', 'Project Name', 'Team', 'billing', 'startDate', 'endDate', 'PM', 'Team Lead', 'Client Lead', 'Valid From', 'Valid To']]
#proj_df.to_csv('proj.csv')

In [5]:
# dim table for project/person
# put together in excel using info from RP

proj_person = pd.read_csv('datasets\personproject.csv')
proj_person.insert(len(proj_person.columns), 'Valid From', pd.to_datetime('2023-03-30').date())
proj_person.insert(len(proj_person.columns), 'Valid To', pd.to_datetime('2262-04-11').date())
#display(proj_person)

In [6]:
# for time dim table

bankhols = requests.get(bankhol_url, headers=tt_key)
bankhols = pd.read_json(bankhols.text)
bankhols = bankhols.loc[(bankhols['countryCode']=='GB-SCT')]
bankhols = bankhols[['id', 'name', 'date']]
bankhols["date"] = pd.to_datetime(bankhols["date"])
bankhols.insert(1, 'non workday bool', True)
bankhols.insert(2, 'non workday count', 1)

start_date = '2023-01-04'
end_date = '2023-12-31'
start_date = pd.to_datetime(start_date).date()
end_date = pd.to_datetime(end_date).date()

time_df = pd.DataFrame({"Date": pd.date_range(start_date, end_date)})
time_df["Day"] = time_df.Date.dt.day_name()
time_df.insert(1, 'non workday bool', False)
time_df.insert(2, 'non workday count', 0)

weekend = ['Saturday', 'Sunday']
time_df['non workday bool'] = np.where(time_df['Day'].isin(weekend), True, time_df['non workday bool'])
time_df['non workday count'] = np.where(time_df['Day'].isin(weekend), 1, time_df['non workday count'])

time_df['non workday bool'] = np.where(time_df['Date'].isin(bankhols["date"].values), True, time_df['non workday bool'])
time_df['non workday count'] = np.where(time_df['Date'].isin(bankhols["date"].values), 1, time_df['non workday count'])

time_df.insert(3, 'Financial Year', '23/24')
time_df['Financial Year'] = time_df['Date'].dt.to_period('Q-JUN').dt.qyear.apply(lambda x: str(x-1) + "/" + str(x))
time_df.insert(4, 'Financial Month', time_df['Date'].dt.month.map(lambda mth: mth + 5 if mth <7 else mth -6)) 
time_df.insert(len(time_df.columns), 'Valid From', pd.to_datetime('2023-03-30').date())
time_df.insert(len(time_df.columns), 'Valid To', pd.to_datetime('2262-04-11').date())
#time_df

In [7]:
# for absences/holidays dim table
# need to adjust for Polish bank hols

hols = requests.get(hol_url, headers=tt_key)
hol_df = pd.read_json(hols.text)

holidays = pd.json_normalize(hol_df.holidays)
absences = holidays.loc[(holidays['leaveType'] == 'Holiday') + (holidays['leaveType'] == 'Sick Leave') + (holidays['leaveType'] == 'Unpaid Leave')
                     + (holidays['leaveType'] == 'Parental Leave') + (holidays['leaveType'] == 'Jury Duty') 
                     + (holidays['leaveType'] == 'Used Bank Hol') + (holidays['leaveType'] == 'Training')]
absences = absences[['startDate','endDate', 'userId', 'userName','leaveType']]
absences.startDate = pd.to_datetime(absences.startDate)
absences.endDate = pd.to_datetime(absences.endDate)
absences = absences.reset_index()
absences = absences.drop(columns=['index'])
absences_df = pd.DataFrame(columns = ['Date', 'startDate', 'endDate', 'userName', 'leaveType'])
for i in range(len(absences)-1):
    for date in pd.date_range(absences.startDate[i], absences.endDate[i]):
        abs = pd.DataFrame([[date, absences.userName[i], absences.leaveType[i], absences.startDate[i], absences.endDate[i]]], columns = ['Date', 'userName', 'leaveType', 'startDate', 'endDate'])
        absences_df = pd.concat([absences_df, abs])  
absences_df.insert(3, 'Absent?', True)
absences_df = absences_df.rename(columns={'userName':'name'})
absences_df.insert(len(absences_df.columns), 'Valid From', pd.to_datetime('2023-03-30').date())
absences_df.insert(len(absences_df.columns), 'Valid To', pd.to_datetime('2262-04-11').date())
absences_df = absences_df.sort_values(by='Date')
#absences_df.to_csv("hols_2023.csv")

In [8]:
# for holidays page of dash app
hols_page = absences_df.merge(proj_person[['Client', 'Project Name','name']],how = 'left', on=['name'])
hols_page['Project'] = hols_page['Client'] + ' ' + hols_page['Project Name']
#hols_page.to_csv("hols_2023.csv")

In [9]:
# Fact Table

fact = pd.merge(time_df,person_df,how='outer')
fact = fact.merge(absences_df,how = 'left', on=['Date', 'name'])
fact = fact.merge(proj_person,how = 'left', on=['name'])

fact['Billing?'] = np.where(fact['billing'].isna() , False, True)
fact['Billing?'] = np.where(fact['non workday bool'] == True , False, fact['Billing?'])

fact['Project'] = fact['Client'] + ' ' + fact['Project Name']
fact['Time Allocation'] = np.where(fact['Time Allocation'].isna() , 8, fact['Time Allocation'])

for i in range(len(fact)):
    if fact['Absent?'][i] != True:
        fact['Absent?'][i] = False 

fact.insert(len(fact.columns), 'Revenue Daily', fact['Time Allocation']/8 * fact['Day Rate_y'])
fact = fact[['Date', 'name', 'departmentName', 'Absent?', 'non workday bool', 'Billing?',   'Time Allocation', 'Project', 'Revenue Daily']]
#display(fact)

C:\Users\OrlaithO'Conner\AppData\Local\Temp\ipykernel_42556\3153937720.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact['Absent?'][i] = False


In [10]:
# Table to be included in overview page of Dash app
# need to fix Effort column - everyone set to 100% effort right now

overview_table = person_df[['name']]
overview_table = overview_table.merge(proj_person[['name', 'Client', 'Project Name']],how = 'left', on=['name'])
overview_table = overview_table.merge(proj_df[['Client', 'Project Name', 'endDate']],how = 'left', on=['Client', 'Project Name'])
overview_table.insert(len(overview_table.columns), 'Effort', '100%')
#overview_table.to_csv('overview_table.csv', index=False)

In [11]:
#for leave summary by month table in Dash app
#need to add

hol_url2 = "https://app.timetastic.co.uk/api/holidays?Start=2023-01-01&End=2023-02-28"
hols = requests.get(hol_url2, headers=tt_key).json()

days = hols["holidays"]

while hols["nextPageLink"]:
    hols = requests.get(hols["nextPageLink"], headers=tt_key).json()
    days.extend(hols["holidays"])

days = pd.json_normalize(days)
days.startDate = pd.to_datetime(days.startDate)
days.endDate = pd.to_datetime(days.endDate)


days_df = pd.DataFrame(columns = ['Date', 'userName', 'leaveType'])
for i in range(len(days)):
    for date in pd.date_range(days.startDate[i], days.endDate[i]):
        abs = pd.DataFrame([[date, days.userName[i], days.leaveType[i]]], columns = ['Date', 'userName', 'leaveType'])
       
        days_df = pd.concat([days_df, abs], ignore_index =True)

holidays = days_df.loc[(days_df['leaveType'] == 'Holiday')]
holidays_total = holidays.leaveType.count()
holidays_jan = holidays.loc[(holidays['Date'].dt.month == 1)].leaveType.count()
holidays_feb = holidays.loc[(holidays['Date'].dt.month == 2)].leaveType.count()


nova_house = days_df.loc[(days_df['leaveType'] == 'Nova House')]
nova_house_total = nova_house.leaveType.count()
nova_house_jan = nova_house.loc[(nova_house['Date'].dt.month == 1)].leaveType.count()
nova_house_feb = nova_house.loc[(nova_house['Date'].dt.month == 2)].leaveType.count()

wfh = days_df.loc[(days_df['leaveType'] == 'WFH')]
wfh_total = wfh.leaveType.count()
wfh_jan = wfh.loc[(wfh['Date'].dt.month == 1)].leaveType.count()
wfh_feb = wfh.loc[(wfh['Date'].dt.month == 2)].leaveType.count()

sick_leave = days_df.loc[(days_df['leaveType'] == 'Sick Leave')]
sick_leave_total = sick_leave.leaveType.count()
sick_leave_jan = sick_leave.loc[(sick_leave['Date'].dt.month == 1)].leaveType.count()
sick_leave_feb = sick_leave.loc[(sick_leave['Date'].dt.month == 2)].leaveType.count()
leave_summary = pd.DataFrame({' ': ['Nova House', 'WFH', 'Holidays', 'Sick Leave'], 'Year To Date' : [nova_house_total, wfh_total, holidays_total, sick_leave_total], 
                              'January' : [nova_house_jan, wfh_jan, holidays_jan, sick_leave_jan], 
                              'February': [nova_house_feb, wfh_feb, holidays_feb, sick_leave_feb]
})

leave_summary#['January']#.to_csv("leave_summary.csv")

,,Year To Date,January,February
0,Nova House,401,211,190
1,WFH,403,222,181
2,Holidays,176,67,62
3,Sick Leave,24,7,17


In [12]:
# roughwork to create loop for data in leave summary page 
months = [1,2,3,4,5,6,7,8,9,10,11,12]
#for i in months:
#    nova_house_jan = nova_house.loc[(nova_house['Date'].dt.month == i)].leaveType.count(Userna)
#print(nova_house_jan)

In [13]:
# Gantt chart for home page of dash app

gantt_data = pd.read_csv('dash_csv/personproject.csv')
gantt_data.startDate = pd.to_datetime(gantt_data.startDate)
gantt_data.endDate = pd.to_datetime(gantt_data.endDate)
gantt_data['colour'] = 'projects'

absences_df = pd.read_csv('dash_csv/hols_2023.csv')
absences_df.Date = pd.to_datetime(absences_df.Date)
absences_df['colour'] = 'holidays'

gantt_data = pd.concat([gantt_data, absences_df])

# manipulating fact table to use in home page gantt chart
x = fact
x['startDate'] = x['Date']
x['endDate'] = x['Date'] + datetime.timedelta(days=1)
x['colour'] = 'weekend/bank holiday'

non_work = x.loc[(x['non workday bool'] == True)]
#non_work.to_csv('wkends_bankhols_2023.csv')
gantt_data = pd.concat([gantt_data, non_work])
fig = px.timeline(gantt_data, x_start='startDate', x_end='endDate', y = 'name', color="colour", color_discrete_sequence=['#B82E2E', '#316395', 'rgb(204,204,204)'])
fig.update_layout(yaxis={"dtick":1},margin={"t":0,"b":0},height=1000)

c:\project\project-planning-tool\.venv\lib\site-packages\plotly\express\_core.py:1074: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  return pd.Series(x).reset_index(drop=True)
c:\project\project-planning-tool\.venv\lib\site-packages\plotly\express\_core.py:1074: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  return pd.Series(x).reset_index(drop=True)
